In [3]:
from transformers import MarianMTModel, MarianTokenizer, AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# Original financial text in Russian
financial_text_russian = ["Компания X объявила о рекордных прибылях за последний квартал.",
                          "Компания X объявила о рекордных убытках за последний квартал.",
                          "Компания X имеет показатели без изменений.",
                          "Электромобильный стартап Arrival экс-главы Yota уйдет из России.",
                          "Шрёдер отклонил предложение войти в совет директоров «Газпрома»",
                          "Шельф берут в разработку // Генподрядчиком «Газпрома» на море может стать компания Андрея Патрушева",
                          "Чистая прибыль 'РусГидро' по РСБУ за 1 полугодие выросла на 17%",
                          "Финский производитель шин Nokian Tyres решил уйти из России",
                          "Федун ушел с поста вице-президента ЛУКОЙЛа на пенсию",
                          "Промсвязьбанк укрепил свои позиции в топ-10 российских банков по портфелю кредитов организациям."
                          ]


translation_model_name = "Helsinki-NLP/opus-mt-ru-en"
translation_model = MarianMTModel.from_pretrained(translation_model_name)
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)

inputs = translation_tokenizer(financial_text_russian, return_tensors="pt", padding=True, truncation=True)
outputs = translation_model.generate(**inputs)
translated_text = translation_tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("Translated Text:", translated_text)

# Choose the FinBERT model and tokenizer for classification
finbert_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Tokenize the translated text for FinBERT
inputs = finbert_tokenizer(translated_text, padding = True, truncation = True, return_tensors='pt')

# Make predictions with FinBERT
outputs = finbert_model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
# print(predictions)


positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline': financial_text_russian,
         'En Headline': translated_text,
         "Positive": positive,
         "Negative": negative, 
         "Neutral": neutral}
      
df = pd.DataFrame(table, columns=["Headline", 'En Headline', "Positive", "Negative", "Neutral"])

df['real_score'] =  df['Positive'] - df['Negative']

df

Translated Text: ['Company X has announced record profits for the last quarter.', 'Company X announced record losses over the last quarter.', 'Company X has no change.', 'The electric motor startup of the Arrival ex-head Yota will leave Russia.', 'Schroeder declined an offer to join the Gazprom Board of Directors.', "Shelf is being developed / Gazproma's contractor at sea may be Andrei Patrushev's company.", 'The net profit of RusHidro on ASHI increased by 17% in the first half of the year', 'The Finnish tyre manufacturer Nokian Tyres decided to leave Russia.', 'Fedun retired as Vice President of Lucoil.', 'Promsbank has strengthened its position in the top 10 of Russian banks on the loan portfolio to organizations.']


,Headline,En Headline,Positive,Negative,Neutral,real_score
0,Компания X объявила о рекордных прибылях за по...,Company X has announced record profits for the...,0.948517,0.025460,0.026023,0.923057
1,Компания X объявила о рекордных убытках за пос...,Company X announced record losses over the las...,0.009817,0.972690,0.017493,-0.962874
2,Компания X имеет показатели без изменений.,Company X has no change.,0.024528,0.030658,0.944815,-0.006130
3,Электромобильный стартап Arrival экс-главы Yot...,The electric motor startup of the Arrival ex-h...,0.035713,0.033495,0.930791,0.002218
4,Шрёдер отклонил предложение войти в совет дире...,Schroeder declined an offer to join the Gazpro...,0.043275,0.024247,0.932478,0.019028
5,Шельф берут в разработку // Генподрядчиком «Га...,Shelf is being developed / Gazproma's contract...,0.033618,0.024472,0.941910,0.009145
6,Чистая прибыль 'РусГидро' по РСБУ за 1 полугод...,The net profit of RusHidro on ASHI increased b...,0.954949,0.014268,0.030784,0.940681
7,Финский производитель шин Nokian Tyres решил у...,The Finnish tyre manufacturer Nokian Tyres dec...,0.016718,0.811226,0.172056,-0.794508
8,Федун ушел с поста вице-президента ЛУКОЙЛа на ...,Fedun retired as Vice President of Lucoil.,0.019642,0.112515,0.867843,-0.092873
9,Промсвязьбанк укрепил свои позиции в топ-10 ро...,Promsbank has strengthened its position in the...,0.959419,0.017457,0.023124,0.941961
